# Measuring behaviour from cameras and accelerometer

In [ ]:
# Imports
import sys
from pathlib import Path
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import actipy  # for reading in accelerometer data

from PIL import Image

sys.path.append("scripts")
# Local scripts
import autographer
from sensorplot import ImageData, ScalarData, VectorData, SensorPlot
from annotate import notebook_annotation

# 1. Prepare camera and accelerometer data

## Camera processing
Plug in the Autographer wearable camera to the computer. 

In [ ]:
# Check the Autographer pops up, i.e. no "No such file or directory" errors
!ls /Volumes/Autographer/DATA

In [ ]:
# Download the data
autographer.downloadData("/Volumes/Autographer/", "raw_data/camera/")

In [ ]:
# If the above was successful probably want to delete the camera data
# autographer.deleteCameraData("/Volumes/Autographer/") # uncomment to run

In [ ]:
# List of image paths and the times from these paths
#             "20231114_182809"
time_format = "%Y%m%d_%H%M%S"

def get_img_times(paths):
    return [datetime.strptime(path.parts[-1][17:32], time_format) for path in paths]

small_img_paths = list(Path("raw_data/camera/small").glob("*.JPG"))
small_img_times = get_img_times(small_img_paths)

tuples = list(zip(small_img_paths, small_img_times))
tuples.sort(key=lambda x: x[0])

small_img_paths, small_img_times = zip(*tuples)

In [ ]:
n_imgs_to_show = 10
n_rows = 2
n_cols = 5

plt.figure(figsize=(20, 10))

for i, (img_path, img_time) in enumerate(zip(small_img_paths[:n_imgs_to_show], small_img_times[:n_imgs_to_show]), 1):
    plt.subplot(n_rows, n_cols, i)
    plt.imshow(Image.open(img_path))
    plt.title(img_time.strftime("%Hh%Mm%Ss"))
    plt.axis("off")

plt.tight_layout()
plt.show()

## Accelerometer processing
Plug in the Axivity

In [ ]:
# Check the Autographer pops up, i.e. something like AX317_43923 under /Volumes
!ls /Volumes

In [ ]:
%%bash
# Copy the .CWA file to raw_data/accelerometer
mkdir raw_data/accelerometer
mv /Volumes/AX317_43923/CWA-DATA.CWA raw_data/accelerometer/CWA-DATA.CWA
ls raw_data/accelerometer

In [ ]:
# Accelerometer reading in
ax3_data, info = actipy.read_device(
    "raw_data/accelerometer/CWA-DATA.CWA",
    lowpass_hz=20,
    calibrate_gravity=True,
    detect_nonwear=True,
    resample_hz=30,
)

In [ ]:
# Plot the acceleromter data
ax3_data[["x", "y", "z"]].plot()
plt.show()

## Visualise the data together

In [ ]:
# img
image_datetimes = np.array(small_img_times, dtype=np.datetime64)
image_paths = np.array(small_img_paths)

# axivity
sensor_datetimes = ax3_data.index.to_numpy()
accelerometer_readings = ax3_data[["x", "y", "z"]].to_numpy()
light_readings = ax3_data["light"].to_numpy()
temperature_readings = ax3_data["temperature"].to_numpy()

In [ ]:
start_time = image_datetimes[50]
duration = np.timedelta64(30, "s")

print(f"Looking at data from {str(start_time)[11:19]} to {str(start_time + duration)[11:19] }")

sensor_data = [
    ImageData("Camera", image_datetimes, image_paths, plot_x_ticks=True, img_zoom=0.22),
    ScalarData("Temperature", sensor_datetimes, temperature_readings, plot_x_ticks=False),
    ScalarData("Light", sensor_datetimes, light_readings, plot_x_ticks=False),
    VectorData("Accelerometer", sensor_datetimes, accelerometer_readings, plot_x_ticks=10, dim_names=["x", "y", "z"]),
]

sv = SensorPlot(sensor_data)
print(sv) 

In [ ]:
fig, ax = sv.plot_window(start_time, duration)
plt.show() 

# Annotate the image data

We put together a simple function using maptlotlib to label the image data inline in this jupyter notebook.
This is implemented in the `notebook_annotation` function.    
    
The following commands are used to navigate:
- "help"/"h" - display the commands
- "next"/. - move to the next N images (if there are any left, but only jumping one image along)
- "prev"/, - move to the previous N images (if there are any left, but only jumping one image along)
- "copy/ c" - copy the current annotation to the next image, and display the next N images
- "quit/ q" - quit the loop, saving the annotations to the numpy array

Importantly, you have to come up with a schema: a list of labels you would like to apply to to describe the activities in  each image. 

To make annotation faster, you also define shortcuts for each label, so that you can just enter the shortcut as opposed to the whole label.

In [ ]:
label_dir_name = "activites" # change this to something more descriptive

schema = { # come up with a better schema
    # shortcut: long name
    "s": "Sedentary",
    "l": "Light",
    "m": "MVPA",
}

notebook_annotation(
    label_dir_name, 
    schema, 
    image_paths, 
    image_datetimes,
    imgs_to_display = 5,
    save_freq = 10,
    figsize=(30, 10),
)